In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import plotly.express as px
import plotly.graph_objects as go

In [73]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# Data Exploration

In [74]:
train.info()

In [75]:
train.head()

In [76]:
train.select_dtypes(np.number).hist(bins=50,figsize=(30,20))
plt.show()

# Features Selection

In [77]:
corr_matrix = train.select_dtypes(np.number).corr()
corr = corr_matrix['SalePrice'].sort_values(ascending=False)
print(corr)
indexNames = corr[abs(corr.values) < 0.4].index.values
indexNames = np.setdiff1d(indexNames, ['Id','MSSubClass'])
#print(len(indexNames), indexNames)
#train.drop(indexNames , inplace=True, axis = 1)
#print(len(train.columns))
#test.drop(indexNames , inplace=True, axis = 1)
#print(train.shape[1],test.shape[1])

### Combining Train & Test Data
 Filling in null values

In [78]:
y_target = train['SalePrice']
test_ids = test.Id
train_v0 = train.drop(['Id','SalePrice'], axis = 1)
test_v0 = test.drop('Id',axis = 1)
data_v0 = pd.concat([train_v0,test_v0],axis = 0)
data_v0

In [79]:
data_v0.info()

In [80]:
y_target

# Data Cleaning
MSSubClass is a categroical column represented as numberic

In [81]:
data_v0['MSSubClass'] = data_v0['MSSubClass'].astype(str)
data_v0.MSSubClass.dtype

In [82]:
cat_mode_cols = [
    'MasVnrType', 
    'MSZoning', 
    'Functional',
    'Utilities',
    'Exterior2nd',
    'KitchenQual', 
    'Electrical', 
    'Exterior1st',
    'SaleType'   
]
for col in cat_mode_cols:
    data_v0[col].fillna(data_v0[col].mode()[0],inplace = True)

cat_None_cols =  [
    'Alley',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'FireplaceQu',
    'FireplaceQu',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'GarageType',
    'PoolQC',
    'Fence',
    'MiscFeature'
]   

for col in cat_None_cols:
    data_v0[col].fillna('None',inplace = True)

In [83]:
ordinal_cols     = {'GarageFinish'  : {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
                    'GarageQual'    : {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'GarageCond'    : {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'BsmtQual'      : {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'BsmtCond'      : {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'BsmtExposure'  : {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
                    'BsmtFinType1'  : {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
                    'BsmtFinType2'  : {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
                    'ExterQual'     : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'ExterCond'     : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'HeatingQC'     : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'KitchenQual'   : {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
                    'Electrical'    : {'Mix': 1, 'FuseP': 2, 'FuseF': 3, 'FuseA': 4, 'SBrkr': 5},
                    'CentralAir'    : {'N': 0, 'Y': 1},
                    'PavedDrive'    : {'N': 1, 'P': 2, 'Y': 3}
                    }

data_v0.replace(ordinal_cols, inplace=True)

Display missing value table

In [84]:
def display_missing(train,cols):
    mis_val = train.isna().sum().sort_values(ascending=False)
    mis_val_per = (mis_val / len(train) *100).sort_values(ascending=False).round(1)
    mis_val_table = pd.concat([mis_val,mis_val_per], axis = 1,keys =['# Missing Values','% Total Missing'])
    return mis_val_table.head(cols)

In [85]:
display_missing(data_v0.select_dtypes('object'),22)

# Feature & Target Transformation 

In [86]:
data_v1 = data_v0.copy()

Kurtosis is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution. That is, data sets with high kurtosis tend to have heavy tails, or outliers. Data sets with low kurtosis tend to have light tails, or lack of outliers.

In [87]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.distplot(data_v1['LotArea'], kde=True, fit=norm)
plt.title("Without Log Transform")

plt.subplot(1, 2, 2)
sns.distplot(np.log1p(data_v1['LotArea']), kde=True, fit=norm,color='red')
plt.xlabel("Log LotArea ")
plt.title("With Log Transform")

plt.show()

In [88]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.distplot(data_v1['GrLivArea'], kde=True, fit=norm)
plt.title("Without Log Transform")

plt.subplot(1, 2, 2)
sns.distplot(np.log1p(data_v1['GrLivArea']), kde=True, fit=norm,color='darkblue')
plt.xlabel("Log GrLivArea ")
plt.title("With Log Transform")

plt.show()

In [89]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.distplot(data_v1['TotalBsmtSF'], kde=True, fit=norm)
plt.title("Without Log Transform")

plt.subplot(1, 2, 2)
sns.distplot(np.log1p(data_v1['TotalBsmtSF']), kde=True, fit=norm,color='darkblue')
plt.xlabel("Log TotalBsmtSF ")
plt.title("With Log Transform")

plt.show()

In [90]:
def display_skew_kurt(df,cols):
    skew= df.select_dtypes(np.number).skew()
    abs_skew = abs(skew)
    kurt = df.select_dtypes(np.number).kurt()
    skew_kurt_table = pd.concat([skew,abs_skew,kurt], axis = 1,
                                keys =['Skew','Absolute Skew','Kurtosis']).sort_values("Skew",ascending = False)
    skew_kurt_table['Skewed'] = skew_kurt_table['Absolute Skew'].apply(lambda x: True if x >= 0.5 else False)
    return skew_kurt_table

In [91]:
skew_kurt_df = display_skew_kurt(data_v1,20)

In [92]:
for col in skew_kurt_df.query("Skewed == True").index.values:
    data_v1[col] = np.log1p(data_v1[col])

Cosine Transformation:

Since the sales in summer are always higher than winter we will give higher priority to June and its surroundings and less to Jan and december

In [93]:
data_v1['MoSold'] = (-np.cos(5.236) * data_v1['MoSold'])

Target Transformation

In [94]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.distplot(y_target, kde=True, fit=norm)
plt.title("Without Log Transform")

plt.subplot(1, 2, 2)
sns.distplot(np.log(y_target), kde=True, fit=norm,color='darkblue')
plt.xlabel("Log SalePrice")
plt.title("With Log Transform")

plt.show()

In [95]:
y_target_log = np.log(y_target)

# Encoding Categorical Columns

In [96]:
data_v2 = data_v1.copy()

In [97]:
data_v2 = pd.get_dummies(data_v2)
data_v2

# Scaling

In [98]:
data_v3= data_v2.copy()

In [99]:
scaler = MinMaxScaler()
data_v3 = pd.DataFrame(scaler.fit_transform(data_v2), columns = data_v3.columns)
data_v3.head()

In [100]:
imputer = KNNImputer(n_neighbors=5)
data_v3 = pd.DataFrame(imputer.fit_transform(data_v3),columns = data_v3.columns)

In [101]:
data_v3.isna().any()

In [102]:
data_v3.isna().sum()

# Split Data

In [103]:
data_v4 = data_v3.copy()

In [104]:
train_final = data_v3.loc[:train_v0.index.max(), :].copy()
test_final = data_v3.loc[train_v0.index.max() + 1:, :].reset_index(drop=True).copy()

# Model Comparison

In [105]:
X_train, X_val, y_train, y_val =train_test_split(train_final, y_target ,train_size=0.8, test_size=0.2,random_state=0)

Random Forest Model

In [106]:
rf = RandomForestRegressor(n_estimators=500, max_depth=10)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_val)
rf.score(X_train,y_train)
print('RMSE:', mean_squared_error(np.log(y_val), np.log(pred_rf),squared=False))

In [107]:
forest_preds = rf.predict(test_final)


Gradient Boosting Regressor

In [108]:
GBR_model = GradientBoostingRegressor(learning_rate=0.01, n_estimators=1000, max_depth=4)
GBR_model.fit(X_train, y_train)
predicted_GBR = GBR_model.predict(X_val)
GBR_model.score(X_train, y_train)
print('RMSE:', mean_squared_error(np.log(y_val), np.log(predicted_GBR),squared=False))

In [109]:
GBR_preds = GBR_model.predict(test_final)

# Submit

In [110]:
submission = pd.concat([test_ids, pd.Series(GBR_preds, name='SalePrice')], axis=1)
submission

In [111]:
submission.to_csv('./submission.csv', index=False, header=True)